## Work process:
* Reduce photo sizes in attachment
* Update columns to match the requirements in ShortList storymap template

#### Columns need to be updated:

* __PIC_URL__ : retrieve Attachment link for pop-up purpose
* __PLACENUMSL__: display order number based on ranking of Date (ASC)
* __DESC1__: convert Date(1514989955000) to Text(2018:01:03 06:32:35)

In [1]:
import arcgis, pandas, requests, PIL.Image, os
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from getpass import getpass
from PIL import Image

In [2]:
password = open("C:\\Users\\kimvo\\OneDrive - King County\\PAM\\getpass.txt").readline()
gis = GIS("http://kingcounty.maps.arcgis.com/home", "kimvo_kingcounty", password)

In [3]:
# Search for all hosted feature layer
from arcgis import features
layer = FeatureLayer('https://services.arcgis.com/Ej0PsM5Aw677QF1W/arcgis/rest/services/PhotoPoint/FeatureServer/0')
data = layer.query()
data.df.head()

,CreationDate,Creator,DESC1,DESC2,Date_Time,EditDate,Editor,GlobalID,Name,OBJECTID,PIC_URL,PLACENUMSL,TAB_NAME,THUMB_URL,SHAPE
0,1533576485342,kimvo_kingcounty,2018:01:03 06:32:35,None,1514989955000,1533576485342,kimvo_kingcounty,725ff055-b30c-4546-a68a-22e7c21dd81b,63rd_Wastewater_0.jpg,1,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,4.0,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."
1,1533576485342,kimvo_kingcounty,2018:01:03 06:33:01,None,1514989981000,1533576485342,kimvo_kingcounty,fb043e21-af4e-4865-a4d9-df3df775d4c9,63rd_Wastewater_1.jpg,2,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,5.0,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."
2,1533576485342,kimvo_kingcounty,2018:01:04 07:16:29,None,1515078989000,1533576485342,kimvo_kingcounty,a16e31f1-8922-42d0-8a9b-87b79225e846,63rd_Wastewater_10.jpg,3,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,13.0,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."
3,1533576485342,kimvo_kingcounty,2018:01:03 06:33:16,None,1514989996000,1533576485342,kimvo_kingcounty,9b3f0c45-c6ce-4768-b691-545f09b1b43d,63rd_Wastewater_2.jpg,4,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,6.0,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."
4,1533576485342,kimvo_kingcounty,2018:01:03 06:33:37,None,1514990017000,1533576485342,kimvo_kingcounty,e6274bce-7cbc-402d-9a3f-d401b25fdd0a,63rd_Wastewater_3.jpg,5,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,7.0,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."


In [4]:
properties = layer.properties
properties

{
  "allowGeometryUpdates": true,
  "hasM": false,
  "name": "Photo Point",
  "supportsRollbackOnFailureParameter": true,
  "tileMaxRecordCount": 8000,
  "attachmentProperties": [
    {
      "name": "name",
      "isEnabled": true
    },
    {
      "name": "size",
      "isEnabled": true
    },
    {
      "name": "contentType",
      "isEnabled": true
    },
    {
      "name": "keywords",
      "isEnabled": true
    }
  ],
  "useStandardizedQueries": false,
  "indexes": [
    {
      "fields": "OBJECTID",
      "name": "PK__PHOTOPOI__F4B70D85EA1B49DC",
      "isUnique": true,
      "description": "",
      "isAscending": true
    },
    {
      "fields": "Shape",
      "name": "user_1368.PHOTOPOINT_PHOTO_POINT_Shape_sidx",
      "isUnique": false,
      "description": "",
      "isAscending": true
    },
    {
      "fields": "GlobalID",
      "name": "GlobalID_idx",
      "isUnique": true,
      "description": "",
      "isAscending": true
    },
    {
      "fields": "CreationDat

In [14]:
fields_to_change = ["PIC_URL","THUMB_URL","DESC1","PLACENUMSL"]
for field in properties.fields:
    if field.name in fields_to_change:
        field.editable = False

In [15]:
layer.manager.update_definition(properties)

RuntimeError: Token Required
Token Required
(Error Code: 499)

In [7]:
# Find oid's that need to be updated
oids = []
for i in data:
    if (i.attributes['PIC_URL']) == None:
        oids.append(i.attributes['OBJECTID'])
oids

[80]

#### Use Attachment Manager class to retrieve photo link

In [8]:
am = arcgis.features.managers.AttachmentManager(layer)

In [9]:
# Reduce photo size in attachment if width > 1126 
# Then replace it with the new smaller one
for i in oids:
    obj = am.search(where='1=1', object_ids=str(i))[0]
    attachment_id = obj['ID']
    obj_id = obj['PARENTOBJECTID']
    img_data = requests.get(obj['DOWNLOAD_URL']).content
    
    # Remove existing file
    if os.path.isfile("NewPhoto.jpg"):
        os.remove('NewPhoto.jpg')
    with open('NewPhoto.jpg', 'wb') as handler:
        handler.write(img_data)
    
    # Open photo, check original size
    img = Image.open('NewPhoto.jpg')
    size = img.size
    if size[0] > 1126:
        print("Reducing size for feature (OBJECTID):", obj_id)
        print("Current size:", size)
        basewidth = 1126
        wpercent = (basewidth/float(img.size[0]))
        hsize = int((float(img.size[1])*float(wpercent)))
        img = img.resize((basewidth,hsize), Image.ANTIALIAS)
        img.save('NewPhoto.jpg', 'JPEG')
        img.close()
        new_img = Image.open('NewPhoto.jpg')
        print("New size:", new_img.size)
        new_img.close()
        # Update new photo to AGO layer
        am.delete(obj_id, attachment_id)
        am.add(obj_id, 'NewPhoto.jpg')
    img.close()

Reducing size for feature (OBJECTID): 80
Current size: (1280, 960)
New size: (1126, 844)


In [10]:
# Get photo links, store them in a dict (OBJECTID -> https...)
# Update to data (json)
for i in oids:
    obj = am.search(where='1=1', object_ids=str(i))[0]
    for j in data:
        if j.attributes['OBJECTID'] == obj['PARENTOBJECTID']:
            j.attributes["PIC_URL"] = obj['DOWNLOAD_URL'].split("?")[0]
data.df.head()

,CreationDate,Creator,DESC1,DESC2,Date_Time,EditDate,Editor,GlobalID,Name,OBJECTID,PIC_URL,PLACENUMSL,TAB_NAME,THUMB_URL,SHAPE
0,1533576485342,kimvo_kingcounty,2018:01:03 06:32:35,None,1514989955000,1533576485342,kimvo_kingcounty,725ff055-b30c-4546-a68a-22e7c21dd81b,63rd_Wastewater_0.jpg,1,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,4.0,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."
1,1533576485342,kimvo_kingcounty,2018:01:03 06:33:01,None,1514989981000,1533576485342,kimvo_kingcounty,fb043e21-af4e-4865-a4d9-df3df775d4c9,63rd_Wastewater_1.jpg,2,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,5.0,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."
2,1533576485342,kimvo_kingcounty,2018:01:04 07:16:29,None,1515078989000,1533576485342,kimvo_kingcounty,a16e31f1-8922-42d0-8a9b-87b79225e846,63rd_Wastewater_10.jpg,3,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,13.0,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."
3,1533576485342,kimvo_kingcounty,2018:01:03 06:33:16,None,1514989996000,1533576485342,kimvo_kingcounty,9b3f0c45-c6ce-4768-b691-545f09b1b43d,63rd_Wastewater_2.jpg,4,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,6.0,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."
4,1533576485342,kimvo_kingcounty,2018:01:03 06:33:37,None,1514990017000,1533576485342,kimvo_kingcounty,e6274bce-7cbc-402d-9a3f-d401b25fdd0a,63rd_Wastewater_3.jpg,5,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,7.0,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."


In [11]:
# Update PLACENUMSL (display number)
myDict = {'Wastewater':[],'Stormwater':[],'Airport':[],'FMD':[],'Non-KC Property':[],'Other KC Property':[],'Parks':[],'Transportation':[]}
for i in data:
    for key in myDict:
        if i.attributes['TAB_NAME'] == key:
            myDict[key].append(i.attributes['Date_Time'])
for key in myDict:
    myDict[key].sort()
# Edit data
for i in data:
    for key in myDict:
        if i.attributes['TAB_NAME'] == key:
            for j in myDict[key]:
                if i.attributes['Date_Time'] == j:
                    i.attributes['PLACENUMSL'] = myDict[key].index(j)
data.df.head()

,CreationDate,Creator,DESC1,DESC2,Date_Time,EditDate,Editor,GlobalID,Name,OBJECTID,PIC_URL,PLACENUMSL,TAB_NAME,THUMB_URL,SHAPE
0,1533576485342,kimvo_kingcounty,2018:01:03 06:32:35,None,1514989955000,1533576485342,kimvo_kingcounty,725ff055-b30c-4546-a68a-22e7c21dd81b,63rd_Wastewater_0.jpg,1,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,4,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."
1,1533576485342,kimvo_kingcounty,2018:01:03 06:33:01,None,1514989981000,1533576485342,kimvo_kingcounty,fb043e21-af4e-4865-a4d9-df3df775d4c9,63rd_Wastewater_1.jpg,2,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,5,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."
2,1533576485342,kimvo_kingcounty,2018:01:04 07:16:29,None,1515078989000,1533576485342,kimvo_kingcounty,a16e31f1-8922-42d0-8a9b-87b79225e846,63rd_Wastewater_10.jpg,3,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,13,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."
3,1533576485342,kimvo_kingcounty,2018:01:03 06:33:16,None,1514989996000,1533576485342,kimvo_kingcounty,9b3f0c45-c6ce-4768-b691-545f09b1b43d,63rd_Wastewater_2.jpg,4,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,6,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."
4,1533576485342,kimvo_kingcounty,2018:01:03 06:33:37,None,1514990017000,1533576485342,kimvo_kingcounty,e6274bce-7cbc-402d-9a3f-d401b25fdd0a,63rd_Wastewater_3.jpg,5,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,7,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."


In [12]:
# Convert and update date (Date -> Text)
for i in data:
    date_str = time.strftime('%m/%d/%Y %H:%M:%S', time.gmtime((i.attributes['Date_Time'])/1000.0))
    i.attributes['DESC1'] = date_str
data.df.head()

,CreationDate,Creator,DESC1,DESC2,Date_Time,EditDate,Editor,GlobalID,Name,OBJECTID,PIC_URL,PLACENUMSL,TAB_NAME,THUMB_URL,SHAPE
0,1533576485342,kimvo_kingcounty,01/03/2018 14:32:35,None,1514989955000,1533576485342,kimvo_kingcounty,725ff055-b30c-4546-a68a-22e7c21dd81b,63rd_Wastewater_0.jpg,1,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,4,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."
1,1533576485342,kimvo_kingcounty,01/03/2018 14:33:01,None,1514989981000,1533576485342,kimvo_kingcounty,fb043e21-af4e-4865-a4d9-df3df775d4c9,63rd_Wastewater_1.jpg,2,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,5,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."
2,1533576485342,kimvo_kingcounty,01/04/2018 15:16:29,None,1515078989000,1533576485342,kimvo_kingcounty,a16e31f1-8922-42d0-8a9b-87b79225e846,63rd_Wastewater_10.jpg,3,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,13,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."
3,1533576485342,kimvo_kingcounty,01/03/2018 14:33:16,None,1514989996000,1533576485342,kimvo_kingcounty,9b3f0c45-c6ce-4768-b691-545f09b1b43d,63rd_Wastewater_2.jpg,4,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,6,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."
4,1533576485342,kimvo_kingcounty,01/03/2018 14:33:37,None,1514990017000,1533576485342,kimvo_kingcounty,e6274bce-7cbc-402d-9a3f-d401b25fdd0a,63rd_Wastewater_3.jpg,5,https://services.arcgis.com/Ej0PsM5Aw677QF1W/a...,7,Wastewater,None,"{'spatialReference': {'latestWkid': 3857, 'wki..."


In [13]:
# Update data to AGO layer
myList = []
for i in data:
    myList.append(i)
for i in myList:
    if i.attributes['OBJECTID'] in oids:
        update_result = layer.edit_features(updates=[i])
        print("Successfully updated feature (OBJECTID):", i.attributes['OBJECTID'])

Successfully updated feature (OBJECTID): 80
